# Pretrained Model, classification, Elastic Net

In [1]:
import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/Innolab/Github') 

Mounted at /content/drive


In [2]:
from src.preprocessing.preprocessing_matlab_files import preprocess_mat_files
from src.preprocessing.data_preparation import prepare_data

In [3]:
matlab_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/Matlab Files" # Enter the directory for the matlab files
excel_path = "/content/drive/MyDrive/Colab Notebooks/Innolab/DELCODE_dataset.xlsx" # Enter the directory for the corresponding excel sheet
write_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/write" # ...
export_file = False # rename to export file
preprocessing_type = 'aggregation' 

df = preprocess_mat_files(matlab_dir = matlab_dir, excel_path = excel_path, write_dir = write_dir,
                           export_file = export_file, preprocessing_type = preprocessing_type, statistic = 'max')

loading files
Starting Preprocessing
Creating Final Dataset
Done!


In [4]:
df

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,IDs,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
0,1,0a8d02f2b,11,66,17.08.2016,0,2,17,0.054016,0.0,1.0,1.442019,0.518327,0.768837,0.781428,0.798942,0.713009,0.569728,0.693034,1.493739,1.055437,1.217551,0.83579,0.699596,1.090191,0.921686,1.499958,1.446869,1.173337,0.854638,1.044583,1.215168,1.374804,0.932106,0.708553,1.14529,1.064851,1.162719,0.580244,1.430655,1.042193,1.244207,0.820703,0.885204,1.26936,1.249158,1.370669
1,2,0a71a953d,17,72,30.03.2015,0,1,20,-0.468749,1.0,2.0,1.052224,0.695097,0.608619,0.617755,0.78139,0.700752,0.661261,0.716709,1.791651,1.182725,1.42357,0.831551,0.855352,1.033703,1.231956,1.824582,1.567818,1.511543,0.75816,1.150066,0.907262,1.392219,1.032674,0.913651,1.212448,1.11951,1.315684,0.75934,0.923772,0.930864,1.534952,0.690841,0.953771,1.211397,1.204627,1.371878
2,3,0a61339db,11,72,13.05.2015,1,1,16,-0.093521,0.0,3.0,0.849411,0.564845,0.555714,0.461625,0.495856,0.788707,0.573609,0.586259,0.976541,0.613206,0.851963,0.370562,0.57159,0.787161,1.130506,1.008474,1.047959,0.770432,0.493032,0.782779,0.576682,1.325337,1.001475,0.600156,0.875494,1.110678,1.021635,0.491159,0.861457,0.908652,0.917063,0.923844,0.699996,0.932172,1.02339,1.326019
3,4,0b28aed58,17,76,18.01.2016,0,1,20,0.466027,0.0,4.0,1.108932,0.911937,0.672048,0.726071,0.781558,0.692417,0.640137,0.889349,1.832284,0.747189,1.01545,0.591824,0.752035,0.702183,1.012694,1.641574,1.338708,1.412591,0.839167,1.014339,1.173346,1.393465,1.042414,0.605472,1.01288,1.523996,1.277929,0.684653,1.290996,1.010103,1.071911,0.774992,0.901711,0.969546,1.222428,1.318915
4,5,0c1c5ae77,8,64,12.03.2015,1,1,13,1.452099,0.0,5.0,0.902319,0.762414,0.702308,0.746809,0.942794,0.745581,0.647355,0.882981,1.391585,0.816195,0.855742,0.694862,0.70731,0.793479,1.369496,1.184785,1.120838,1.121519,0.565561,0.868133,0.722518,1.071553,0.803008,0.689007,0.988444,0.808636,1.108088,0.713629,1.128626,0.83524,1.140538,0.879643,0.923432,1.153777,0.889666,1.244347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,906,f816d269a,16,84,2017-11-21,2,3,18,-2.388095,0.0,906.0,1.417167,0.612684,0.789715,0.470563,0.879898,0.911326,0.945266,0.965765,1.673786,0.815731,1.171023,0.80011,0.492837,0.885967,0.996097,1.460167,1.441553,1.374246,0.559355,0.923871,0.806196,1.305496,1.182395,0.56674,1.025421,0.828947,1.038595,0.71342,1.064049,0.880292,0.858801,0.996523,0.70373,0.887294,1.088491,1.377185
905,907,f98419539,5,71,2014-09-11,2,3,13,-1.860675,1.0,907.0,1.107081,0.489421,0.620626,0.396967,1.084753,0.643209,0.861978,1.207757,1.509016,0.881523,0.977685,0.640263,0.565235,0.739145,1.026662,1.510382,1.111873,1.226397,0.74193,0.906277,1.366042,1.258578,0.829937,0.639695,1.048475,0.877682,1.387487,0.621841,1.000504,1.59777,1.281182,0.760985,0.987812,1.168594,0.962629,1.500568
906,908,fade343c8,8,78,2015-11-23,2,3,8,-1.868184,1.0,908.0,0.769773,0.426069,0.621418,0.445918,0.537328,0.650858,0.611799,0.927391,0.908209,0.627183,0.848678,0.461005,0.47463,0.637235,0.649359,0.84264,0.94299,0.799259,0.518328,0.541375,0.779953,0.904409,0.608589,0.659208,0.865961,0.8663,0.836314,0.502671,0.726753,0.808973,0.914337,0.60809,0.578736,0.769957,1.071035,1.132849
907,909,fbc8a48a0,14,61,2017-03-20,2,3,9,-2.432351,0.0,909.0,0.778697,0.526224,0.526895,0.466178,0.669391,0.619619,0.61346,0.634253,1.335476,0.497276,0.845021,0.599592,0.627642,0.881996,1.127493,1.301591,1.258621,0.890437,0.537687,0.614483,0.689756,0.954526,0.812523,0.400247,1.182646,0.834384,1.158202,0.579332,0.986103,1.063459,0.906869,0.812011,0.861273,1.008168,1.016017,1.177611


In [5]:
classification = True
columns_drop = ["ConnID", "Repseudonym", "siteid", "visdat", "MEM_score", "Apoe", "IDs"]
target = "prmdiag"
y_0 = [0]
y_1 = [2, 3]
train_size = 0.8
seed = 123
split = True

In [6]:
# preparation of data
ytrain, Xtrain, ytest, Xtest = prepare_data(data = df, classification = classification,
                         columns_drop = columns_drop, target = target, y_0 = y_0, y_1 = y_1,
                         train_size = train_size, seed = seed, split = split)


pd.concat([ytest, Xtest], axis = 1) # test data

,y,age,sex,edyears,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
30,0.0,1.774171,-1.149848,-0.727818,0.163757,-0.338126,0.459585,-0.360658,2.058142,1.050582,-0.907339,1.569250,0.946112,1.542920,1.970527,0.163311,0.075981,0.346990,0.508634,-0.014880,2.190952,2.299845,0.854297,-0.291741,2.065399,1.804841,1.876862,0.228156,-0.659644,-0.803193,-0.389016,0.794313,-1.028216,0.178669,0.249747,1.216502,-0.639508,-1.035159,-0.838652,-0.700295
792,1.0,1.106080,-1.149848,2.001953,-0.033308,-0.227664,1.162322,1.113561,1.489831,2.237732,0.760112,1.949408,-0.213580,1.217957,0.123145,0.069830,1.108820,-1.648488,-1.267524,0.493783,1.459223,0.761662,0.779859,0.784094,-0.117944,0.082139,-0.186261,0.447555,0.116522,-0.124063,0.846454,1.557650,0.020949,0.007323,2.337990,0.767719,-0.696286,0.161360,0.666999,-0.561452
497,0.0,0.103945,0.469414,-0.386597,0.439230,0.440738,-0.090151,-0.297719,0.074079,-0.983427,-0.824975,-0.319430,1.266720,1.807186,0.827144,1.198536,-0.606682,0.113746,0.069349,1.126678,-0.015250,0.613261,0.062604,-0.365666,-0.201288,0.755642,-0.274603,-0.535607,0.506483,-0.705343,-0.094591,-0.013787,1.557440,0.042340,-0.715538,-1.199579,1.190572,-0.543423,-0.144509,-0.784626
594,1.0,1.440126,-1.149848,1.660732,1.534454,-0.387284,0.025842,-0.667629,-0.375181,0.225472,-0.207985,-0.347269,0.947464,-0.157632,-0.054867,0.242194,0.094891,-0.785289,0.570002,0.221366,0.346491,1.402550,0.257170,0.149040,0.673554,-0.174354,-0.031061,-0.396728,-0.615558,-0.752574,0.678316,0.567964,0.090666,0.905811,-0.207682,-0.259227,-0.806472,-1.153935,-0.252780,-0.934211
712,0.0,0.772035,0.469414,-0.727818,-0.134981,-0.346822,1.162464,2.245811,0.825598,-0.262183,0.301556,0.147583,1.282695,-1.760230,-0.015379,-1.212446,0.303418,-0.374237,0.789797,-0.254495,0.633452,0.668683,1.991100,0.677925,-0.701252,0.911676,0.994828,-0.604417,0.519896,0.766998,1.287687,3.017440,-0.489546,-0.584536,0.760950,0.356260,-1.108321,-0.198817,0.456356,0.159567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,0.0,-0.898191,-1.149848,0.978289,0.443637,1.872170,0.551966,1.025298,0.964995,1.041528,-0.583438,-0.294577,1.695008,1.685811,2.769752,1.103215,0.759702,-2.145083,1.129832,0.637552,0.782834,-0.824450,1.442273,1.086016,1.157462,-1.152236,-0.369673,0.446362,-0.447871,-0.725725,0.957963,-0.616812,0.188484,-0.202809,0.491247,0.132509,0.205564,-0.387312,0.143047,0.402143
332,1.0,0.605013,0.469414,-0.386597,-0.996979,-0.418095,-0.206642,0.142159,-1.139797,-0.167679,-0.884224,-0.584542,0.541010,0.850780,0.497233,0.439308,-1.048602,0.965198,0.394346,-0.317853,-0.159101,0.011640,-0.907288,-0.692079,-1.113609,-0.712052,-1.012215,-0.732022,-0.360339,0.233724,-0.205487,-0.273429,-0.826423,-0.111770,0.595765,-2.005556,-1.417270,0.565044,1.822959,2.280686
228,1.0,0.103945,0.469414,2.001953,3.411606,7.315733,6.241412,9.407605,7.983162,6.881312,5.854150,5.431292,1.771714,5.309901,4.621591,6.039597,8.332120,5.530643,4.540116,5.257080,4.018833,3.660477,7.318383,4.953733,6.084329,4.046204,4.631492,8.041012,5.637224,5.202351,4.108078,5.586037,3.497419,5.023274,1.889511,4.219665,6.959934,4.169481,3.128176,1.350189
397,1.0,2.108216,-1.149848,-0.727818,-0.456632,-0.842708,0.067686,-1.449529,1.147076,-0.817466,-1.134622,-1.194903,-0.589226,-0.526026,-0.086601,-1.380976,-0.730645,-0.071215,0.259977,0.058143,-0.061391,-1.263390,-0.794191,-1.891557,-0.199321,-0.495291,-1.326515,0.323905,-0.958656,-0.481643,-1.664476,-0.190819,-0.717524,-1.251367,-0.583544,-0.309402,0.479185,0.703637,-0.024642,0.471876


# Train Model
Evaluate AUC and accuracy on test data

In [9]:
from src.models.pipeline_elastic_net import model_elastic_net
from sklearn.metrics import accuracy_score, roc_auc_score

In [10]:
model = model_elastic_net(X_train = Xtrain, y_train = ytrain, 
                          classification = True, 
                          n_alphas_logreg = 10, 
                          l1_ratios_logreg= np.linspace(0,  1, 11).tolist(), 
                          cv_logreg = 5,
                          max_iter = 500) 

In [11]:
predictions = model.predict(Xtest)
accuracy_score(ytest, predictions)

0.6914893617021277

In [12]:
pred_prob = model.predict_proba(Xtest)
auc = roc_auc_score(ytest, pred_prob[:, 1])
auc

0.6533882783882784

In [13]:
model.l1_ratio_

array([0.1])

In [14]:
model.C_

array([0.04641589])

# Save Model

In [15]:
import pickle

In [16]:
filename ='pretrained_elnet_class_agg_max.sav'
pickle.dump(model, open(filename, 'wb'))

# Visualisation Feature Importance and Coeffs (evtl Ridge)

In [ ]:
# load model
model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/Innolab/pretrained_elnet_class_agg_max.sav", 'rb'))